In [2]:
import os
import sys
from pathlib import Path

file = Path(os.path.abspath('')).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import import_ipynb
import json
from collections import defaultdict
import syntaxCheck as sync
import semanticCheck as smtc

In [1]:
class validate1:
    
    def __init__(self, tar_file):
        self.tar_file = tar_file
        self.extension = os.path.splitext(self.tar_file)[1]
        self.NAME = None
        self.error_msg = None
        self.bool = False
        
        #define a dictionary storing the checking functions in value
        dictionary = {
            "File existence": sync.existence_check,
            "File size": sync.file_size_check, 
            "Metadata file existence":sync.metadata_check,
            "Json check": sync.string_value_check,
            "Json-ld check": sync.check_context,
            "Self descriptor check": smtc.self_descriptor_check, 
            "Direct property check": smtc.direct_property_check,
            "Referencing check":smtc.referencing_check, 
            "Encoding check":smtc.encoding_check,
            "Web-based data entity check": smtc.webbased_entity_check,
            "Person entity check": smtc.person_entity_check,
            "Organization check": smtc.organization_check,
            "Scripts and workflow check":smtc.scripts_and_workflow_check
        }
        
        #initialize the check result and a list to store the checking function
        self.functions = []
        self.result = defaultdict(list)
        
        with open ("check_list.txt") as file: 
            check_list = list(map(lambda s: s.strip(), file.readlines()))
        
        ### store the functions in a dictionary
        for x in check_list:
            if x in dictionary:
                self.functions.append(dictionary[x])
        
        for i in range(len(self.functions)):
            self.result[list(dictionary.keys())[i].rstrip("\n")].append(None)

    ### get the name, error message and boolean result of check funciton
    def get_result(self, NAME, error_msg, bool):
        self.NAME = NAME
        self.error_msg = error_msg
        self.bool = bool
        
    def update_output(self, NAME, *check_result):
        self.result[NAME][0] = check_result[1]
        if check_result[0] != []:
            self.result[NAME].append(check_result[0])
    
    def validator(self):
        for methods in self.functions: 
            check_result = methods(self.tar_file, self.extension)
            self.get_result(check_result[0], check_result[1], check_result[2])
            if (self.NAME == "File existence" or self.NAME == "File_size" or self.NAME == "Metadata file existence") and self.bool == False:
                self.update_output(self.NAME, self.error_msg, self.bool)
                break
            self.update_output(self.NAME, self.error_msg, self.bool)
        output = json.dumps(self.result, indent = 4, separators=(',', ': '), sort_keys = False)

        print(output)
        
        with open('result.json', 'w') as file:
            file.write(json.dumps(self.result, indent = 4, separators=(',', ': '),sort_keys = False))
        